# Speech Recognition

Ziele der heutigen Übung:
- Kennenlernen der SpeechRecognition Bibliothek
- Transkribieren einer Spracheingabe mit der Google Web Speech API
- Verstehen von Erkennerergebnissen eines Spracherkenners
- Erleben von Herausforderungen der Spracherkennung


## Speech Recognition API

Heute lernen wir eine Bibliothek kennen, welche verschiedene Spracherkenner und online APIs anbindet. Wir verwenden hierfür die Google Speech API als Cloud Service und Vosk als Spracherkenner, welcher lokal mit begrenzten Ressourcen arbeitet.

Installiere folgende Bibliothek: https://pypi.org/project/SpeechRecognition/

In [ ]:
pip install SpeechRecognition

In [ ]:
import speech_recognition as sr
print(f"We use speech_recognition version: {sr.__version__}")

Um die Eingabe des Mikrofons aufzunehmen und an den Spracherkenner weiter zu leiten, nutzen wir die Bibliothek PyAudio. Installiere PyAudio mit pip. Je nach Betriebssystem kann es erforderlich sein zusätzlieh Pakete mit homebrew (macOS) oder apt (ubuntu) zu installieren. Du findest sicherlich mit Google eine Lösung:

In [ ]:
pip install pyaudio

In [ ]:
from pyaudio import PyAudio

p = PyAudio()
try:
    print(p.get_default_input_device_info())
except:
    print("No mics available")

## Transkribieren einer sprachlichen Äußerung mit der Google Web Speech API (cloud-basiert)

Folgendes Tutorial gibt gute Hinweise, wie wir die SpeechRecognition Bibliothek nutzen: https://realpython.com/python-speech-recognition/

In [36]:
recognizer = sr.Recognizer()

Erfasse eine sprachliche Äußerung mit dem Mikrofon:

In [ ]:
with sr.Microphone() as source:
    print("Speak something...")
    audio_data = recognizer.listen(source)
    print("Audio data recorded.")

Nun schicken wir die erfasste Äußerung an die Google Web Speech API. Sofern die API nicht erreichbar ist, wirft der ```recognizer``` einen ```RequestError```. Sofern in dem übergebenen ```audio_data``` keine sprachliche Äußerung enthalten ist gibt der ```recognizer``` einen ```UnknownValueError``` aus. Prüfe auf beide Exceptions und geben dem Nutzer ein entsprechendes Feedback auf der Konsole aus.

In [ ]:
try:
    text = recognizer.recognize_google(audio_data)
    print("You said:", text)
except sr.RequestError:
    print("Error: Could not access Google Web Speech API;")
except sr.UnknownValueError:
    print("Sorry, I do not understand")


Neben Englisch kann die Google Web Speech API auch Deutsch. Passe den Code entsprechend an, dass Deutsch erkannt wird:

In [ ]:
try:
    text = recognizer.recognize_google(audio_data, language='de-DE')
    print("Du hast gesagt: ", text)
except sr.RequestError:
    print("Fehler: Google Web Speech API ist nicht erreichbar")
except sr.UnknownValueError:
    print("Sorry, Ich verstehe die Äußerung nicht")

Wir werden in der nächsten Vorlesung genauer auf die Technik der Spracherkennung schauen. Heute möchten wir uns jedoch bereits mehrer Erkennerergebnisse und die jeweilige Konfidenz anschauen. Mit dem Parameter ```show_all``` gibt der ```recognizer``` mehrere Alternativen als JSON aus. Integriere dies in deine Erkennung (Tipp: für den weiteren Verlauf ist es ratsam dies als Funktion zu implementieren):

In [ ]:
import json

def recognize_google(audio_data: any):
    try:
        text = recognizer.recognize_google(audio_data, language='de-DE', show_all=True)
        print(f"[Google] Recognition results: {json.dumps(text, sort_keys=True, indent=4)}")
    except sr.RequestError:
        print("[Google] Fehler: Google Web Speech API ist nicht erreichbar")
    except sr.UnknownValueError:
        print("[Google] Sorry, Ich verstehe die Äußerung nicht")
recognize_google(audio_data)

## Spracherkennung ohne Cloud und Internetanbindung

Gerade als Ingenieure haben wir oftmals mit einer fehlenden Internetverbindung in Produktionsumgebungen, begrenzten Hardware-Ressourcen, Datenschutzanforderungen oder was auch immer zu tun, das es verhindert Daten in die Cloud zu schicken. Für solche Fälle möchten wir uns mit https://alphacephei.com/vosk/ einen Spracherkenner anschauen, der einerseits lokal und andererseits auch mit begrenzen Ressourcen zurecht kommt (z.B. Raspberry PI). Installiere die Bibliothek mit ```pip```

In [ ]:
pip install vosk

Vosk unterstützt verschiedene Sprachen und bietet Sprachmodelle in unterschiedlicher Größe an: https://alphacephei.com/vosk/models. Starte mit einem kleinen Modell (vosk-model-small-**), lade dieses herunter und nutze die Funktion ```recognizer.recognize_vosk```, um deine Äußerung zu transkribieren. Auch hier ist es hilfreich dies als Funktion zu implementieren...


In [ ]:
with sr.Microphone() as source:
    print("Speak something...")
    audio_data = recognizer.listen(source)
    print("Audio data recorded.")

In [ ]:
def recognize_vosk(audio_data: any):
    try:
        text = recognizer.recognize_vosk(audio_data, language='de')
        text_json = json.loads(text)
        print("[Vosk] You said:", text_json['text'])
    except sr.RequestError:
        print("[Vosk] Error: Could not access Google Web Speech API;")
    except sr.UnknownValueError:
        print("[Vosk] Sorry, I do not understand")
recognize_vosk(audio_data)

## Herausforderungen der Spracherkennung

Wir haben nun zwei Erkenner angebunden, einerseits ein großes Modell in der Cloud und andererseits ein kleines Modell, welches lokal erkennt. Doch wie gut funktionieren diese? Und wo sind die Unterschiede? Nehme dir etwas Zeit und versuche verschiedene Äußerungen aus, variiere dabei zum Beispiel:
- Anwendungsfälle
- Namen von Personen, Städten, Restaurants, ...
- Sprechgeschwindigkeit
- Lautstärke
- Deutlich vs. undeutliche Aussprache
- Dialekt (kann Google Fränkisch?)
- Kommandos vs. natürlichsprachlich
- Lange vs. kurze Äußerungen
- Englische Begriffe in einem deutschen Satz (oder anders rum)
- Umgangssprache

Speichere dir hierfür die mit dem Mikrofon aufgenommenen Daten in eine Variable ```audio_data``` und gebe diese Daten an ```recognize_google(audio_data, ..``` und ```recognize_vosk(audio_data, ..```. Hier ist es nun praktisch auf die zuvor erstellen Funktionen zurückzugreifen ;-)

In [ ]:
with sr.Microphone() as source:
    print("Speak something...")
    audio_data = recognizer.listen(source)
    print("Audio data recorded.")
    
    recognize_google(audio_data)
    recognize_vosk(audio_data)